In [2]:
import requests
import pandas as pd
import json
from dotenv import load_dotenv
import os
import time

In [3]:
api_key = {}
try:
    with open('api_key.json') as file:
        api_key = json.load(file)
except FileNotFoundError:
    print("Error: credentials.json file not found.")

In [4]:
url = "https://v1.basketball.api-sports.io/standings?league=12&season=2022-2023"

payload = {}
headers = {
    'x-rapidapi-key': api_key['x-rapidapi-key'],
    'x-rapidapi-host': 'v1.basketball.api-sports.io'
}

In [10]:
# The team_id_grab() function grabs the teams id,name,conference and conference position and stores it in a DataFrame
    
def team_ids():
    # request.request sends a request of a specified method to the url (in this case the method is GET)
    response = requests.request("GET", url, headers=headers, data=payload)

    # json.loads deserialises a JSON object to a standard python object (good for parsing)
    # also it turns nulls to a None python object
    standings = json.loads(response.text)

    # team_lvl (team level) grabs the section of the returned data that we need to parse through to collect 
    # info on the team
    team_lvl = standings['response'][0]
       
    # Creating a DataFrame to store the team API ids
    team_ids_df = pd.DataFrame(columns = ['Team', 'Team ID', 'Conference', 'Conference Position'])
     
    # Creating variables that hold the requests daily limit remaining (100 per day) 
    # and the per minute requests remaining (10 per minute)
    daily_calls_remaining = response.headers['x-ratelimit-requests-remaining']
    perMinute_calls_remaining = response.headers['X-RateLimit-Remaining']
    
    print(f'Daily calls remaining = {daily_calls_remaining}')
    print(f'Calls left per minute = {perMinute_calls_remaining}\n')

    df_row = 0

    team_id_list = []
    
    for team in team_lvl:
        if team['group']['name'] == "Western Conference" or team['group']['name'] == "Eastern Conference":
            t_id = int(team['team']['id'])
            t_name = team['team']['name']
            t_conference = team['group']['name']
            t_position = str(team['position'])

            team_ids_df.loc[df_row] = [t_name, t_id, t_conference, t_position]

            team_id_list.append(t_id)
            df_row += 1
            
    return team_id_list, team_ids_df

team_ids = team_ids()
team_ids_df = team_ids[1]
team_ids_list = team_ids[0]

print(team_ids_list)
print(team_ids_df)

def team_ppg():
    team_ppg_df = pd.DataFrame(columns = ['Team', 'Team ID', 'PPG For', 'PPG Against'])

    df_row = 0

    for id in team_ids_list:
        t_id = str(id)
        # API URL for NBA team data via api-basketball.com
        url = "https://v1.basketball.api-sports.io/statistics?season=2022-2023&team="+t_id+"&league=12"

        payload = {}
        headers = {
            'x-rapidapi-key': api_key['x-rapidapi-key'],
            'x-rapidapi-host': 'v1.basketball.api-sports.io'
        }

        daily_calls_remaining = 1
        perMinute_calls_remaining = 1

        # if statement to make sure I don't go over the API's free version daily and per minunte limit
        if perMinute_calls_remaining > 0 and daily_calls_remaining > 0: 
            response = requests.request("GET", url, headers=headers, data=payload)
            # request.request sends a request of a specified method to the url (in this case the method is GET)

            # Parses the JSON
            team_statistics = json.loads(response.text)
            # Use the print statment below to check the layout of the data pulled with the GET request
            # print(json.dumps(team_statistics, indent=4))

            daily_calls_remaining = int(response.headers['x-ratelimit-requests-remaining'])
            perMinute_calls_remaining = int(response.headers['X-RateLimit-Remaining'])

            ppg_for = team_statistics["response"]["points"]["for"]["average"]["all"]
            ppg_against = team_statistics["response"]["points"]["against"]["average"]["all"]
            team_name = team_statistics["response"]["team"]["name"]

            team_ppg_df.loc[df_row] = [team_name, id, ppg_for, ppg_against]
            df_row += 1

            
            print('=====================================================================================')

            print(team_ppg_df)

            print('=====================================================================================')
            
            print('Daily calls remaining = ' + str(daily_calls_remaining))
            print('Calls left per minute = ' + str(perMinute_calls_remaining), '\n')

            time.sleep(7)
            # Sleep for 7 seconds to avoid having more than 10 calls per minute as I am using the free API  
        else:
            break

    return team_ppg_df

Daily calls remaining = 96
Calls left per minute = 9

[139, 146, 157, 155, 141, 144, 150, 145, 149, 152, 138, 160, 156, 142, 158, 148, 133, 154, 137, 151, 134, 147, 132, 159, 136, 161, 153, 143, 135, 140]
                      Team  Team ID          Conference Conference Position
0           Denver Nuggets      139  Western Conference                   1
1        Memphis Grizzlies      146  Western Conference                   2
2         Sacramento Kings      157  Western Conference                   3
3             Phoenix Suns      155  Western Conference                   4
4    Golden State Warriors      141  Western Conference                   5
5     Los Angeles Clippers      144  Western Conference                   6
6     New Orleans Pelicans      150  Western Conference                   7
7       Los Angeles Lakers      145  Western Conference                   8
8   Minnesota Timberwolves      149  Western Conference                   9
9    Oklahoma City Thunder      152

In [11]:
team_ppg_df = team_ppg()

             Team  Team ID PPG For PPG Against
0  Denver Nuggets      139   115.2       111.2
Daily calls remaining = 95
Calls left per minute = 8 

                Team  Team ID PPG For PPG Against
0     Denver Nuggets      139   115.2       111.2
1  Memphis Grizzlies      146   115.8       112.5
Daily calls remaining = 94
Calls left per minute = 7 

                Team  Team ID PPG For PPG Against
0     Denver Nuggets      139   115.2       111.2
1  Memphis Grizzlies      146   115.8       112.5
2   Sacramento Kings      157   120.0       116.7
Daily calls remaining = 93
Calls left per minute = 6 



KeyboardInterrupt: 